In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F 
from resnet1d import Res34SimSiam, Res34SimSiamSplitHeads, Resnet34Baseline
from dataset import Dataset_per_file, Dataset_whole, Dataset_ori, Dataset_ori_npy
from torch.utils.data import DataLoader
import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
from datetime import datetime
from torch.autograd import Variable
import os
import numpy as np
from tqdm import tqdm
import argparse
import random
from sklearn.metrics import f1_score, accuracy_score, roc_auc_score, precision_recall_curve, auc
import sys
import os

from collections import OrderedDict
import pickle as pkl

In [2]:
def test_epoch(PPG_model, test_loader, test_ds_name, baseline=False, patient_info_path=None):
    with torch.no_grad():
       
        all_targets = None
        PPG_preds = None
        PPG_pred_probs = None
        all_targets = None
        
        PPG_model.eval()
        tstart = datetime.now()

        for batch_idx, (PPG, target) in enumerate(test_loader):

            PPG = PPG.cuda().float()
            target = target.cuda().long()
            
            if baseline:
                _, PPG_out = PPG_model(PPG, PPG)
            else:
                _, _, _, _, _, PPG_out = PPG_model(PPG, PPG)
            
            PPG_predicted = PPG_out.argmax(1)
            PPG_predicted_prob = F.softmax(PPG_out, dim=1)[:, 1]

            if PPG_preds == None:
                PPG_pred_probs = PPG_predicted_prob
                PPG_preds = PPG_predicted
                all_targets = target
            else:
                PPG_preds = torch.cat((PPG_preds, PPG_predicted))
                PPG_pred_probs = torch.cat((PPG_pred_probs, PPG_predicted_prob))
                all_targets = torch.cat((all_targets, target))
        tend = datetime.now()

        precision, recall, thresholds = precision_recall_curve(all_targets.detach().cpu().numpy(), PPG_pred_probs.detach().cpu().numpy())
        pr_auc = auc(recall, precision)

        print(f'[TEST] \tPPG      F1: {round(f1_score(all_targets.detach().cpu().numpy(), PPG_preds.detach().cpu().numpy()), 4)}')
        print(f'[TEST] \tPPG ROC AUC: {round(roc_auc_score(all_targets.detach().cpu().numpy(), PPG_pred_probs.detach().cpu().numpy()), 4)}')
        print(f'[TEST] \tPPG PR  AUC: {round(pr_auc, 4)}')
        rounding = 3
        if patient_info_path is not None:
            print(patient_info_path)
            patient_idx_dict = pkl.load(open(patient_info_path, 'rb'))
            print(patient_idx_dict)
            unique_patients = list(patient_idx_dict.keys())
            all_bt_aurocs = []
            all_bt_auprcs = []
            all_bt_pred_probs = np.asarray([])
            all_bt_targets = np.asarray([])
            for i in tqdm(range(1000)):
                np.random.seed(i)
                random_sample_patients = np.random.choice(unique_patients, len(unique_patients), replace=True)
                sample_idx = []
                random_subsample_idx = []
                for patient in random_sample_patients:
                    random_subsample_idx += patient_idx_dict[patient]
                random_subsample_idx = np.asarray(random_subsample_idx)
                auroc = roc_auc_score(all_targets[random_subsample_idx].detach().cpu().numpy(), PPG_pred_probs[random_subsample_idx].detach().cpu().numpy())
                precision, recall, thresholds = precision_recall_curve(all_targets[random_subsample_idx].detach().cpu().numpy(), PPG_pred_probs[random_subsample_idx].detach().cpu().numpy())
                auprc = auc(recall, precision)

                all_bt_pred_probs = np.concatenate((PPG_pred_probs[random_subsample_idx].detach().cpu().numpy().flatten(), all_bt_pred_probs))
                all_bt_targets = np.concatenate((all_targets[random_subsample_idx].detach().cpu().numpy().flatten(), all_bt_targets))
                all_bt_aurocs.append(auroc)
                all_bt_auprcs.append(auprc)
            all_bt_aurocs = np.asarray(all_bt_aurocs)
            all_bt_auprcs = np.asarray(all_bt_auprcs)
                        
            np.save(f'/labs/hulab/stark_stuff/ppg_ecg_project/data/{test_ds_name}_simsiam_aurocs.npy', all_bt_aurocs)
            np.save(f'/labs/hulab/stark_stuff/ppg_ecg_project/data/{test_ds_name}_simsiam_auprcs.npy', all_bt_auprcs)
            np.save(f'/labs/hulab/stark_stuff/ppg_ecg_project/data/{test_ds_name}_simsiam_targets.npy', all_bt_targets)
            np.save(f'/labs/hulab/stark_stuff/ppg_ecg_project/data/{test_ds_name}_simsiam_pred_probs.npy', all_bt_pred_probs)

            print(f'\t AUROC {round(np.mean(all_bt_aurocs), rounding)} [{round(np.mean(all_bt_aurocs) - 1.96 *  (np.std(all_bt_aurocs) / np.sqrt(len(all_bt_aurocs))) , rounding)} {round(np.mean(all_bt_aurocs) + 1.96 *  (np.std(all_bt_aurocs) / np.sqrt(len(all_bt_aurocs))) , rounding)}]')
            print(f'\t AUPRC {round(np.mean(all_bt_auprcs), rounding)} [{round(np.mean(all_bt_auprcs) - 1.96 *  (np.std(all_bt_auprcs) / np.sqrt(len(all_bt_auprcs))) , rounding)} {round(np.mean(all_bt_auprcs) + 1.96 *  (np.std(all_bt_auprcs) / np.sqrt(len(all_bt_auprcs))) , rounding)}]')


# SimSiam

In [3]:
# MODEL_PATH = '/home/zguo30/ppg_ecg_proj/simsiam/saved_models/corrected_resnet_34_epoch_30_ecglr_0.0001_ppglr_0.0001_lambda_1.0_dim1_512_dim2_128_pred_True_bs=2500/model_2.pt'
# MODEL_PATH = '/home/zguo30/ppg_ecg_proj/simsiam/saved_models/corrected_resnet_34_epoch_30_ecglr_0.0001_ppglr_0.0001_lambda_1.0_dim1_512_dim2_128_pred_True_bs=2500/model_2.pt'
MODEL_PATH = '/home/zguo30/ppg_ecg_proj/proposed/saved_models/corrected_resnet_34_epoch_30_ecglr_0.0001_ppglr_0.0001_lambda_1.0_dim1_512_dim2_128_pred_True_bs=5000/model_1.pt'
# MODEL_PATH = '/home/zguo30/ppg_ecg_proj/simsiam/saved_models/corrected_resnet_34_epoch_30_ecglr_0.0001_ppglr_0.0001_lambda_1.0_dim1_512_dim2_128_pred_True_/model_2.pt'


In [2]:
model = Res34SimSiam(512, 128).cuda()

In [3]:
model.encoder.fc

Sequential(
  (0): Linear(in_features=384, out_features=384, bias=False)
  (1): BatchNorm1d(384, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU(inplace=True)
  (3): Linear(in_features=384, out_features=384, bias=False)
  (4): BatchNorm1d(384, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (5): ReLU(inplace=True)
  (6): Linear(in_features=384, out_features=512, bias=True)
  (7): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
)

In [5]:
state_dict = torch.load(MODEL_PATH) 
new_state_dict = OrderedDict()
for k, v in state_dict.items():
    name = k[7:] #remove 'module'
    new_state_dict[name] = v
    
state_dict = new_state_dict
model.load_state_dict(state_dict)

<All keys matched successfully>

In [6]:
testing_data_label_pairs = [
    # ['data_simband_ecg_2400.npy', 'label_simband.npy', '/labs/hulab/Robust_learning_TESTDATA/simband_patient_idx_dict.pkl'],
    # ['data_simband_ppg_2400.npy', 'label_simband.npy', '/labs/hulab/Robust_learning_TESTDATA/simband_patient_idx_dict.pkl'],
    # ['data_ucla_ecg_2400.npy', 'label_ucla_ecg.npy', '/labs/hulab/Robust_learning_TESTDATA/UCLA_patient_idx_dict.pkl'],
    # ['data_ucla_ppg_2400.npy', 'label_ucla_ppg.npy', '/labs/hulab/Robust_learning_TESTDATA/UCLA_patient_idx_dict.pkl'],
    # ['data_staford_2400.npy', 'label_staford.npy', '/labs/hulab/Robust_learning_TESTDATA/stanford_patient_idx_dict.pkl'],
    
    ['data_simband_goodquality_2400.npy', 'label_simband_goodquality.npy', '/labs/hulab/Robust_learning_TESTDATA/simband_patient_idx_dict.pkl'],
    ['data_simband_badquality_2400.npy', 'label_simband_badquality.npy', '/labs/hulab/Robust_learning_TESTDATA/simband_patient_idx_dict.pkl']
    ['data_ucla_goodquality_2400.npy', 'label_ucla_goodquality.npy', '/labs/hulab/Robust_learning_TESTDATA/UCLA_patient_idx_dict.pkl'],
    ['data_ucla_badquality_2400.npy', 'label_ucla_badquality.npy', '/labs/hulab/Robust_learning_TESTDATA/UCLA_patient_idx_dict.pkl']
    ['data_staford_goodquality_2400.npy', 'label_staford_goodquality.npy', '/labs/hulab/Robust_learning_TESTDATA/stanford_patient_idx_dict.pkl'],
    ['data_staford_badquality_2400.npy', 'label_staford_badquality.npy', '/labs/hulab/Robust_learning_TESTDATA/stanford_patient_idx_dict.pkl']
]

test_data_folder = '/labs/hulab/Robust_learning_TESTDATA/'
for pair in testing_data_label_pairs:
    
    print(pair[0].split('.')[0].split('/')[-1])
    
    x_path = test_data_folder + pair[0]
    y_path = test_data_folder + pair[1]
    
    print(x_path, pair[0].split('.')[0])
    print(y_path)
    test_dataset = Dataset_ori(x_path, y_path)
    testloader = DataLoader(test_dataset, batch_size=2500, shuffle=False, num_workers=0)
    
    test_epoch(model, testloader, pair[0].split('.')[0], patient_info_path=pair[2] if len(pair)==3 else None)

    print()

data_simband_goodquality_2400
(172, 2400) (172,)
/labs/hulab/Robust_learning_TESTDATA/cheng_test/data_simband_goodquality_2400.npy data_simband_goodquality_2400
/labs/hulab/Robust_learning_TESTDATA/cheng_test/label_simband_goodquality.npy
[TEST] 	PPG      F1: 0.9577
[TEST] 	PPG ROC AUC: 1.0
[TEST] 	PPG PR  AUC: 1.0

data_simband_badquality_2400
(685, 2400) (685,)
/labs/hulab/Robust_learning_TESTDATA/cheng_test/data_simband_badquality_2400.npy data_simband_badquality_2400
/labs/hulab/Robust_learning_TESTDATA/cheng_test/label_simband_badquality.npy
[TEST] 	PPG      F1: 0.769
[TEST] 	PPG ROC AUC: 0.8775
[TEST] 	PPG PR  AUC: 0.843

data_ucla_goodquality_2400
(151960, 2400) (151960,)
/labs/hulab/Robust_learning_TESTDATA/cheng_test/data_ucla_goodquality_2400.npy data_ucla_goodquality_2400
/labs/hulab/Robust_learning_TESTDATA/cheng_test/label_ucla_goodquality.npy
[TEST] 	PPG      F1: 0.7649
[TEST] 	PPG ROC AUC: 0.9346
[TEST] 	PPG PR  AUC: 0.7639

data_ucla_badquality_2400
(107690, 2400) (1076

# SimSiam Quality CV

In [4]:
# MODEL_PATH = '/home/zguo30/ppg_ecg_proj/simsiam/saved_models/corrected_resnet_34_epoch_30_ecglr_0.0001_ppglr_0.0001_lambda_1.0_dim1_512_dim2_128_pred_True_bs=2500/model_2.pt'
# MODEL_PATH = '/home/zguo30/ppg_ecg_proj/simsiam/saved_models/corrected_resnet_34_epoch_30_ecglr_0.0001_ppglr_0.0001_lambda_1.0_dim1_512_dim2_128_pred_True_bs=2500/model_2.pt'
MODEL_PATH = '/home/zguo30/ppg_ecg_proj/proposed/saved_models/corrected_resnet_34_epoch_30_ecglr_0.0001_ppglr_0.0001_lambda_1.0_dim1_512_dim2_128_pred_True_bs=5000/model_1.pt'
# MODEL_PATH = '/home/zguo30/ppg_ecg_proj/simsiam/saved_models/corrected_resnet_34_epoch_30_ecglr_0.0001_ppglr_0.0001_lambda_1.0_dim1_512_dim2_128_pred_True_/model_2.pt'
model = Res34SimSiam(512, 128).cuda()
state_dict = torch.load(MODEL_PATH) 
new_state_dict = OrderedDict()
for k, v in state_dict.items():
    name = k[7:] #remove 'module'
    new_state_dict[name] = v
    
state_dict = new_state_dict
model.load_state_dict(state_dict)

<All keys matched successfully>

In [11]:
from sklearn.model_selection import KFold, StratifiedKFold

def test_epoch_qual_cv(PPG_model, x_path, y_path, test_ds_name, baseline=False, patient_info_path=None):
    with torch.no_grad():
       
        rounding = 3
        
        x = np.load(x_path)
        y = np.load(y_path)
        print(x.shape, y.shape)
        skf = StratifiedKFold(n_splits=5)
        AUROCs = []
        AUPRCs = []
        
        for i, (train_index, test_index) in enumerate(skf.split(x, y)):
            print('Split', i, len(train_index))
            x_split = x[train_index]
            y_split = y[train_index]
            
            test_dataset = Dataset_ori_npy(x_split,y_split)
            test_loader = DataLoader(test_dataset, batch_size=2500, shuffle=False, num_workers=0)
                   
            all_targets = None
            PPG_preds = None
            PPG_pred_probs = None
            all_targets = None

            PPG_model.eval()
            tstart = datetime.now()

            for batch_idx, (PPG, target) in enumerate(test_loader):

                PPG = PPG.cuda().float()
                target = target.cuda().long()

                if baseline:
                    _, PPG_out = PPG_model(PPG, PPG)
                else:
                    _, _, _, _, _, PPG_out = PPG_model(PPG, PPG)

                PPG_predicted = PPG_out.argmax(1)
                PPG_predicted_prob = F.softmax(PPG_out, dim=1)[:, 1]

                if PPG_preds == None:
                    PPG_pred_probs = PPG_predicted_prob
                    PPG_preds = PPG_predicted
                    all_targets = target
                else:
                    PPG_preds = torch.cat((PPG_preds, PPG_predicted))
                    PPG_pred_probs = torch.cat((PPG_pred_probs, PPG_predicted_prob))
                    all_targets = torch.cat((all_targets, target))
            tend = datetime.now()

            precision, recall, thresholds = precision_recall_curve(all_targets.detach().cpu().numpy(), PPG_pred_probs.detach().cpu().numpy())
            auprc = auc(recall, precision)
            auroc = roc_auc_score(all_targets.detach().cpu().numpy(), PPG_pred_probs.detach().cpu().numpy())
            
            AUROCs.append(auroc)
            AUPRCs.append(auprc)
        print(AUROCs)
        print(AUPRCs)
        print(f'\t AUROC {round(np.mean(AUROCs), rounding)} {round(np.std(AUROCs), rounding)}')
        print(f'\t AUPRC {round(np.mean(AUPRCs), rounding)} {round(np.std(AUPRCs) , rounding)}')


In [12]:
testing_data_label_pairs = [
    ['data_simband_goodquality_2400.npy', 'label_simband_goodquality.npy', '/labs/hulab/Robust_learning_TESTDATA/simband_patient_idx_dict.pkl'],
    ['data_simband_badquality_2400.npy', 'label_simband_badquality.npy', '/labs/hulab/Robust_learning_TESTDATA/simband_patient_idx_dict.pkl'],
    ['data_ucla_goodquality_2400.npy', 'label_ucla_goodquality.npy', '/labs/hulab/Robust_learning_TESTDATA/UCLA_patient_idx_dict.pkl'],
    ['data_ucla_badquality_2400.npy', 'label_ucla_badquality.npy', '/labs/hulab/Robust_learning_TESTDATA/UCLA_patient_idx_dict.pkl'],
    ['data_staford_goodquality_2400.npy', 'label_staford_goodquality.npy', '/labs/hulab/Robust_learning_TESTDATA/stanford_patient_idx_dict.pkl'],
    ['data_staford_badquality_2400.npy', 'label_staford_badquality.npy', '/labs/hulab/Robust_learning_TESTDATA/stanford_patient_idx_dict.pkl'],
]

test_data_folder = '/labs/hulab/Robust_learning_TESTDATA/cheng_test/'
for pair in testing_data_label_pairs:
    
    print(pair[0].split('.')[0].split('/')[-1])
    
    x_path = test_data_folder + pair[0]
    y_path = test_data_folder + pair[1]
    
    print(x_path, pair[0].split('.')[0])
    print(y_path)
    
    test_epoch_qual_cv(model, x_path, y_path, pair[0].split('.')[0], patient_info_path=pair[2] if len(pair)==3 else None)


data_simband_goodquality_2400
/labs/hulab/Robust_learning_TESTDATA/cheng_test/data_simband_goodquality_2400.npy data_simband_goodquality_2400
/labs/hulab/Robust_learning_TESTDATA/cheng_test/label_simband_goodquality.npy
(172, 2400) (172,)
Split 0 137
Split 1 137
Split 2 138
Split 3 138
Split 4 138
[1.0, 1.0, 1.0, 1.0, 1.0]
[0.9999999999999998, 0.9999999999999998, 0.9999999999999998, 0.9999999999999998, 1.0]
	 AUROC 1.0 0.0
	 AUPRC 1.0 0.0
data_simband_badquality_2400
/labs/hulab/Robust_learning_TESTDATA/cheng_test/data_simband_badquality_2400.npy data_simband_badquality_2400
/labs/hulab/Robust_learning_TESTDATA/cheng_test/label_simband_badquality.npy
(685, 2400) (685,)
Split 0 548
Split 1 548
Split 2 548
Split 3 548
Split 4 548
[0.8508055320804325, 0.860866299113311, 0.888204756730653, 0.8805451594296216, 0.905231123981124]
[0.8083830852047287, 0.8231296906262593, 0.8512331714904134, 0.8472684132523502, 0.8817960515242041]
	 AUROC 0.877 0.019
	 AUPRC 0.842 0.025
data_ucla_goodquality_2

# SimSiam no predictor

In [14]:
MODEL_PATH = '/home/zguo30/ppg_ecg_proj/simsiam/saved_models/epoch_40_ecglr_0.0001_ppglr_0.0001_lambda_1.0_dim1_512_dim2_128_pred_False_no_predictor/model_1.pt'

In [15]:
model = Res34SimSiam(512, 128, predictor=False).cuda()

In [16]:
state_dict = torch.load(MODEL_PATH) 
new_state_dict = OrderedDict()
for k, v in state_dict.items():
    name = k[7:] #remove 'module'
    new_state_dict[name] = v
    
state_dict = new_state_dict
model.load_state_dict(state_dict)

<All keys matched successfully>

In [17]:

test_data_folder = '/labs/hulab/Robust_learning_TESTDATA/'
for f in os.listdir(test_data_folder):
    if '2400' in f and 'sully' not in f:
        
        x_path = test_data_folder + f
        y_path = test_data_folder + 'label_'+'_'.join(f[:-4].split('_')[1:-1]) + '.npy'

        print(f[:-4])
        print(x_path)
        print(y_path)
        test_dataset = Dataset_ori(x_path, y_path)
        testloader = DataLoader(test_dataset, batch_size=2500, shuffle=False, num_workers=0)

        test_epoch(model, testloader)

        print()

data_simband_2400
/labs/hulab/Robust_learning_TESTDATA/data_simband_2400.npy
/labs/hulab/Robust_learning_TESTDATA/label_simband.npy
[TEST] 	PPG      F1: 0.7244
[TEST] 	PPG ROC AUC: 0.9115
[TEST] 	PPG PR  AUC: 0.8661

data_simband_goodquality_2400
/labs/hulab/Robust_learning_TESTDATA/data_simband_goodquality_2400.npy
/labs/hulab/Robust_learning_TESTDATA/label_simband_goodquality.npy
[TEST] 	PPG      F1: 0.9577
[TEST] 	PPG ROC AUC: 0.9996
[TEST] 	PPG PR  AUC: 0.9983

data_simband_badquality_2400
/labs/hulab/Robust_learning_TESTDATA/data_simband_badquality_2400.npy
/labs/hulab/Robust_learning_TESTDATA/label_simband_badquality.npy
[TEST] 	PPG      F1: 0.6944
[TEST] 	PPG ROC AUC: 0.8752
[TEST] 	PPG PR  AUC: 0.8416

data_staford_goodquality_2400
/labs/hulab/Robust_learning_TESTDATA/data_staford_goodquality_2400.npy
/labs/hulab/Robust_learning_TESTDATA/label_staford_goodquality.npy
[TEST] 	PPG      F1: 0.9805
[TEST] 	PPG ROC AUC: 0.9996
[TEST] 	PPG PR  AUC: 0.998

data_staford_badquality_2400

# SimSiam split heads

In [4]:
MODEL_PATH = '/home/zguo30/ppg_ecg_proj/simsiam/saved_models/corrected_resnet_34_epoch_30_ecglr_0.0001_ppglr_0.0001_lambda_1.0_dim1_512_dim2_128_pred_True_splitheads/model_2.pt'

In [5]:
model = Res34SimSiamSplitHeads(512, 128).cuda()

In [6]:
state_dict = torch.load(MODEL_PATH) 
new_state_dict = OrderedDict()
for k, v in state_dict.items():
    name = k[7:] #remove 'module'
    new_state_dict[name] = v
    
state_dict = new_state_dict
model.load_state_dict(state_dict)

<All keys matched successfully>

In [8]:
testing_data_label_pairs = [
    ['data_simband_ecg_2400.npy', 'label_simband.npy'],
    ['data_simband_ppg_2400.npy', 'label_simband.npy'],
    ['data_ucla_ecg_2400.npy', 'label_ucla_ecg.npy', '/labs/hulab/Robust_learning_TESTDATA/UCLA_patient_idx_dict.pkl'],
    ['data_ucla_ppg_2400.npy', 'label_ucla_ppg.npy', '/labs/hulab/Robust_learning_TESTDATA/UCLA_patient_idx_dict.pkl'],
    ['data_staford_2400.npy', 'label_staford.npy', '/labs/hulab/Robust_learning_TESTDATA/stanford_patient_idx_dict.pkl'],
#     ['data_staford_goodquality_2400.npy', 'label_staford_goodquality.npy', '/labs/hulab/Robust_learning_TESTDATA/stanford_patient_idx_dict.pkl'],
#     ['data_staford_badquality_2400.npy', 'label_staford_badquality.npy', '/labs/hulab/Robust_learning_TESTDATA/stanford_patient_idx_dict.pkl']
]

test_data_folder = '/labs/hulab/Robust_learning_TESTDATA/'
for pair in testing_data_label_pairs:
    
    print(pair[0].split('.')[0].split('/')[-1])
    
    x_path = test_data_folder + pair[0]
    y_path = test_data_folder + pair[1]

    print(x_path)
    print(y_path)
    test_dataset = Dataset_ori(x_path, y_path)
    testloader = DataLoader(test_dataset, batch_size=2500, shuffle=False, num_workers=0)

    test_epoch(model, testloader, )

    print()

data_simband_ecg_2400
/labs/hulab/Robust_learning_TESTDATA/data_simband_ecg_2400.npy
/labs/hulab/Robust_learning_TESTDATA/label_simband.npy
[TEST] 	PPG      F1: 0.5464
[TEST] 	PPG ROC AUC: 0.6293
[TEST] 	PPG PR  AUC: 0.5098

data_simband_ppg_2400
/labs/hulab/Robust_learning_TESTDATA/data_simband_ppg_2400.npy
/labs/hulab/Robust_learning_TESTDATA/label_simband.npy
[TEST] 	PPG      F1: 0.8154
[TEST] 	PPG ROC AUC: 0.9243
[TEST] 	PPG PR  AUC: 0.881

data_ucla_ecg_2400
/labs/hulab/Robust_learning_TESTDATA/data_ucla_ecg_2400.npy
/labs/hulab/Robust_learning_TESTDATA/label_ucla_ecg.npy
[TEST] 	PPG      F1: 0.4648
[TEST] 	PPG ROC AUC: 0.7391
[TEST] 	PPG PR  AUC: 0.3608

data_ucla_ppg_2400
/labs/hulab/Robust_learning_TESTDATA/data_ucla_ppg_2400.npy
/labs/hulab/Robust_learning_TESTDATA/label_ucla_ppg.npy
[TEST] 	PPG      F1: 0.6893
[TEST] 	PPG ROC AUC: 0.9262
[TEST] 	PPG PR  AUC: 0.8028

data_staford_2400
/labs/hulab/Robust_learning_TESTDATA/data_staford_2400.npy
/labs/hulab/Robust_learning_TESTDA

# PPG only baseline

In [16]:
MODEL_PATH = '/home/zguo30/ppg_ecg_proj/ppg_only_baseline/saved_models/epoch_40_ppglr_0.0001_lambda_0.9/PPG_best_2.pt'

In [19]:
model = Resnet34Baseline().cuda()
state_dict = torch.load(MODEL_PATH) 
model.load_state_dict(state_dict)

<All keys matched successfully>

In [22]:

test_data_folder = '/labs/hulab/Robust_learning_TESTDATA/'
for f in os.listdir(test_data_folder):
    if '2400' in f:
        
        x_path = test_data_folder + f
        y_path = test_data_folder + 'label_'+'_'.join(f[:-4].split('_')[1:-1]) + '.npy'

        print(f[:-4])
        print(x_path)
        print(y_path)
        test_dataset = Dataset_ori(x_path, y_path)
        testloader = DataLoader(test_dataset, batch_size=2500, shuffle=False, num_workers=0)

        test_epoch(model, testloader, baseline=True)

        print()

data_simband_2400
/labs/hulab/Robust_learning_TESTDATA/data_simband_2400.npy
/labs/hulab/Robust_learning_TESTDATA/label_simband.npy
[TEST] 	PPG F1: 0.7703488372093024
[TEST] 	PPG AUC: 0.9026940360860827

data_simband_goodquality_2400
/labs/hulab/Robust_learning_TESTDATA/data_simband_goodquality_2400.npy
/labs/hulab/Robust_learning_TESTDATA/label_simband_goodquality.npy
[TEST] 	PPG F1: 0.9189189189189189
[TEST] 	PPG AUC: 0.9997868712702472

data_simband_badquality_2400
/labs/hulab/Robust_learning_TESTDATA/data_simband_badquality_2400.npy
/labs/hulab/Robust_learning_TESTDATA/label_simband_badquality.npy
[TEST] 	PPG F1: 0.7524429967426709
[TEST] 	PPG AUC: 0.8664995622092125

data_staford_goodquality_2400
/labs/hulab/Robust_learning_TESTDATA/data_staford_goodquality_2400.npy
/labs/hulab/Robust_learning_TESTDATA/label_staford_goodquality.npy
[TEST] 	PPG F1: 0.9202453987730062
[TEST] 	PPG AUC: 0.9988102683204434

data_staford_badquality_2400
/labs/hulab/Robust_learning_TESTDATA/data_staford_

# Simsiam Adam optim

In [3]:
MODEL_PATH = '/home/zguo30/ppg_ecg_proj/simsiam/saved_models/corrected_resnet_34_epoch_30_ecglr_0.0001_ppglr_0.0001_lambda_1.2_dim1_512_dim2_128_pred_True_bs=5000Adam/model_4.pt'

model = Res34SimSiam(512, 128).cuda()
state_dict = torch.load(MODEL_PATH) 
new_state_dict = OrderedDict()
for k, v in state_dict.items():
    name = k[7:] #remove 'module'
    new_state_dict[name] = v
    
state_dict = new_state_dict
model.load_state_dict(state_dict)

testing_data_label_pairs = [
    ['data_simband_ecg_2400.npy', 'label_simband.npy'],
    ['data_simband_ppg_2400.npy', 'label_simband.npy'],
    ['data_ucla_ecg_2400.npy', 'label_ucla_ecg.npy'],
    ['data_ucla_ppg_2400.npy', 'label_ucla_ppg.npy'],
    ['data_staford_2400.npy', 'label_staford.npy'],
    ['data_staford_goodquality_2400.npy', 'label_staford_goodquality.npy'],
    ['data_staford_badquality_2400.npy', 'label_staford_badquality.npy']
]

test_data_folder = '/labs/hulab/Robust_learning_TESTDATA/'
for pair in testing_data_label_pairs:
    
    print(pair[0].split('.')[0].split('/')[-1])
    
    x_path = test_data_folder + pair[0]
    y_path = test_data_folder + pair[1]

    print(x_path)
    print(y_path)
    test_dataset = Dataset_ori(x_path, y_path)
    testloader = DataLoader(test_dataset, batch_size=2500, shuffle=False, num_workers=0)

    test_epoch(model, testloader)

    print()

data_simband_ecg_2400
/labs/hulab/Robust_learning_TESTDATA/data_simband_ecg_2400.npy
/labs/hulab/Robust_learning_TESTDATA/label_simband.npy
[TEST] 	PPG      F1: 0.6285
[TEST] 	PPG ROC AUC: 0.7267
[TEST] 	PPG PR  AUC: 0.6498

data_simband_ppg_2400
/labs/hulab/Robust_learning_TESTDATA/data_simband_ppg_2400.npy
/labs/hulab/Robust_learning_TESTDATA/label_simband.npy
[TEST] 	PPG      F1: 0.7578
[TEST] 	PPG ROC AUC: 0.9196
[TEST] 	PPG PR  AUC: 0.8796

data_ucla_ecg_2400
/labs/hulab/Robust_learning_TESTDATA/data_ucla_ecg_2400.npy
/labs/hulab/Robust_learning_TESTDATA/label_ucla_ecg.npy
[TEST] 	PPG      F1: 0.7495
[TEST] 	PPG ROC AUC: 0.904
[TEST] 	PPG PR  AUC: 0.7567

data_ucla_ppg_2400
/labs/hulab/Robust_learning_TESTDATA/data_ucla_ppg_2400.npy
/labs/hulab/Robust_learning_TESTDATA/label_ucla_ppg.npy
[TEST] 	PPG      F1: 0.714
[TEST] 	PPG ROC AUC: 0.9164
[TEST] 	PPG PR  AUC: 0.7755

data_staford_2400
/labs/hulab/Robust_learning_TESTDATA/data_staford_2400.npy
/labs/hulab/Robust_learning_TESTDAT

In [4]:
MODEL_PATH = '/home/zguo30/ppg_ecg_proj/simsiam/saved_models/corrected_resnet_34_epoch_30_ecglr_0.0001_ppglr_0.0001_lambda_1.0_dim1_512_dim2_128_pred_True_bs=5000Adam/model_1.pt'

model = Res34SimSiam(512, 128).cuda()
state_dict = torch.load(MODEL_PATH) 
new_state_dict = OrderedDict()
for k, v in state_dict.items():
    name = k[7:] #remove 'module'
    new_state_dict[name] = v
    
state_dict = new_state_dict
model.load_state_dict(state_dict)

testing_data_label_pairs = [
    ['data_simband_ecg_2400.npy', 'label_simband.npy'],
    ['data_simband_ppg_2400.npy', 'label_simband.npy'],
    ['data_ucla_ecg_2400.npy', 'label_ucla_ecg.npy'],
    ['data_ucla_ppg_2400.npy', 'label_ucla_ppg.npy'],
    ['data_staford_2400.npy', 'label_staford.npy'],
    ['data_staford_goodquality_2400.npy', 'label_staford_goodquality.npy'],
    ['data_staford_badquality_2400.npy', 'label_staford_badquality.npy']
]

test_data_folder = '/labs/hulab/Robust_learning_TESTDATA/'
for pair in testing_data_label_pairs:
    
    print(pair[0].split('.')[0].split('/')[-1])
    
    x_path = test_data_folder + pair[0]
    y_path = test_data_folder + pair[1]

    print(x_path)
    print(y_path)
    test_dataset = Dataset_ori(x_path, y_path)
    testloader = DataLoader(test_dataset, batch_size=2500, shuffle=False, num_workers=0)

    test_epoch(model, testloader)

    print()

data_simband_ecg_2400
/labs/hulab/Robust_learning_TESTDATA/data_simband_ecg_2400.npy
/labs/hulab/Robust_learning_TESTDATA/label_simband.npy
[TEST] 	PPG      F1: 0.623
[TEST] 	PPG ROC AUC: 0.6888
[TEST] 	PPG PR  AUC: 0.5651

data_simband_ppg_2400
/labs/hulab/Robust_learning_TESTDATA/data_simband_ppg_2400.npy
/labs/hulab/Robust_learning_TESTDATA/label_simband.npy
[TEST] 	PPG      F1: 0.7963
[TEST] 	PPG ROC AUC: 0.9155
[TEST] 	PPG PR  AUC: 0.8709

data_ucla_ecg_2400
/labs/hulab/Robust_learning_TESTDATA/data_ucla_ecg_2400.npy
/labs/hulab/Robust_learning_TESTDATA/label_ucla_ecg.npy
[TEST] 	PPG      F1: 0.6999
[TEST] 	PPG ROC AUC: 0.9099
[TEST] 	PPG PR  AUC: 0.733

data_ucla_ppg_2400
/labs/hulab/Robust_learning_TESTDATA/data_ucla_ppg_2400.npy
/labs/hulab/Robust_learning_TESTDATA/label_ucla_ppg.npy
[TEST] 	PPG      F1: 0.6742
[TEST] 	PPG ROC AUC: 0.9259
[TEST] 	PPG PR  AUC: 0.7779

data_staford_2400
/labs/hulab/Robust_learning_TESTDATA/data_staford_2400.npy
/labs/hulab/Robust_learning_TESTDAT

# SGD [VAL] Epoch 1 Loss: -1.3191769726601126 simsam loss : -0.9764100327073032, ce loss : 0.3168215469497266, 	PPG F1: 0.9428633667019257, ECG F1: 0.94055694451495 -- Chose SGD


# Adam [VAL] Epoch 1 Loss: -1.3098863723243863 simsam loss : -0.9926099595630468, ce loss : 0.33766677324062605, 	PPG F1: 0.941941411465695, ECG F1: 0.9302179029918973
